In [2]:
from random import random, sample, randint

In [77]:
class BinaryEvolution:
    def __init__(self, number_range=(0, 1024), mutate_rate=3/4, max_flip_rate=1/4, gen_survival_range=(1/4, 1/2), gen_size=20):
        self.min, self.max = number_range
        self.n_bits = len(bin(self.max)) - 2
        self.target = randint(self.min, self.max)
        self.mutate_rate = mutate_rate
        self.max_flip_rate = max_flip_rate
        self.gen_survival_range = gen_survival_range
        self.gen_size = gen_size
        self.population = [bin(randint(self.min, self.max)) for _ in range(self.gen_size)]

        print(f"Trying numbers between {self.min} and {self.max} ({self.n_bits} bits).")
        print(f"We are looking for {self.target} ({bin(self.target)}).")
        print(f"Generation size is {self.gen_size}.")

    def fitness(self, x):
        return -abs(int(x, base=0) - self.target)

    def step(self):
        """Steps one generation ahead."""
        scores = [{"string": x, "fitness": self.fitness(x)} for x in self.population]
        scores.sort(key=lambda x: x["fitness"], reverse=True)

        # If the correct string has been found, return it.
        if scores[0]["fitness"] == 0:
            return scores[0]["string"]

        # Pick a random number of the best individuals.
        min_survival, max_survival = self.gen_survival_range
        random_best_scores = scores[: randint(int(min_survival * self.gen_size), int(max_survival * self.gen_size)) - 1]
        next_generation = [s["string"] for s in random_best_scores]

        # Combine the existing individuals into new ones to fill next generation.
        while self.gen_size > len(next_generation):
            x1, x2 = sample(next_generation, 2)
            next_generation.append(self.combine(x1, x2))

        # Mutate the population (mutate_rate determines how many get mutated).
        for i in range(len(next_generation)):
            if i > int(self.gen_size * self.mutate_rate):
                next_generation[i] = self.mutate(next_generation[i])

        self.population = next_generation

    def combine(self, x1, x2):
        """Combine binary numbers by randomly picking digits from the two numbers."""
        x1, x2 = self.uniform_length(x1), self.uniform_length(x2)
        return "".join([x1[i] if random() < 0.5 else x2[i] for i in range(len(x1))])

    def mutate(self, x):
        """Flips a random number of bits at random indexes up to the limit defined by self.max_change_rate."""
        x = list(self.uniform_length(x))

        n_mut = randint(0, int(self.n_bits * self.max_flip_rate))  # At most int(flip n_bits * max_mut_rate)
        flip_indexes = sample(range(2, self.n_bits + 2), n_mut)  # Get n_mut random indexes (string starts with 0b)

        for i in flip_indexes:
            x[i] = "1" if (x[i] == "0") else "0"

        return "".join(x)

    def uniform_length(self, x):
        """Returns the byte string with leading zeros so the length is uniform."""
        return format(int(x, base=0), f"#0{self.n_bits + 2}b")
        

be = BinaryEvolution()

for i in range(100):
    print(f"Step {i + 1}: best fitness = {be.fitness(be.population[0])}")

    result = be.step()

    if result is not None:
        print(f"{be.target} => {result[2:]} was found after {i + 1} steps.")
        break


Trying numbers between 0 and 1024 (11 bits).
We are looking for 925 (0b1110011101).
Generation size is 20.
Step 1: best fitness = -179
Step 2: best fitness = -11
Step 3: best fitness = -11
Step 4: best fitness = -1
Step 5: best fitness = -1
Step 6: best fitness = -1
Step 7: best fitness = -1
Step 8: best fitness = -1
Step 9: best fitness = -1
Step 10: best fitness = -1
925 => 01110011101 was found after 10 steps.
